# Quantum Square Distance Classifier

Importing what is needed

In [1]:
import numpy as np

from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, measure_partial
from sympy.physics.quantum.gate import HadamardGate, IdentityGate
from sympy import sqrt, simplify

We have two points from Titanic dataset, normalized to one

In [2]:
# price, room , survival
data = np.array([[0.921, 0.390, 1], [0.141, 0.990, 0]])
data

array([[0.921, 0.39 , 1.   ],
       [0.141, 0.99 , 0.   ]])

New passaenger, we want to estimate the survival of

In [3]:
x_point = np.array([0.866, 0.500])
x_point

array([0.866, 0.5  ])

calculate the distance of the new point from the two existing one

In [4]:
dists = []
for i in range(2):
    d = np.linalg.norm(x_point - data[i][0:2])
    dists.append(1.0 - 0.25 * d)

dists = dists / np.sum(dists)
print(dists)

[0.5537045 0.4462955]


Create a state concatenating the values of the passengers plus twice the unknown one
each feature is the phase of one qubit from zero to seven
then we concatenate zero or one if the passenger survived or not

In [5]:
# this is the quantum state where we have:
# Passenger 1 f1 f2 living
# Passenger 2 f1 f2 dead
# Passenger 3 f1 f2 living
# Passenger 3 f1 f2 dead

data = np.array(
    [[0.921, 0.390, 1], [0.141, 0.990, 0], [0.866, 0.500, 1], [0.866, 0.500, 0]]
)

s = 0
for i in range(4):
    for j in range(2):
        # assigning a number between 0 and 7 to each
        b = "{0:03b}".format(i * 2 + j)
        # put one or zero if the passenger survived or not
        b += str(int(data[i][2]))
        s += 1 / sqrt(4) * Qubit(b) * data[i][j]

print(f"s={s}")

data = np.array(
    [[0.921, 0.390, 1], [0.141, 0.990, 0], [0.866, 0.500, 1], [0.866, 0.500, 0]]
)

s = 0
for i in range(4):
    for j in range(2):
        # assigning a number between 0 and 7 to each
        b = "{0:03b}".format(i * 2 + j)
        # put one or zero if the passenger survived or not
        b += str(int(data[i][2]))
        s += 1 / sqrt(4) * Qubit(b) * data[i][j]

print(f"s={s}")

s=0.4605*|0001> + 0.195*|0011> + 0.0705*|0100> + 0.495*|0110> + 0.433*|1001> + 0.25*|1011> + 0.433*|1100> + 0.25*|1110>
s=0.4605*|0001> + 0.195*|0011> + 0.0705*|0100> + 0.495*|0110> + 0.433*|1001> + 0.25*|1011> + 0.433*|1100> + 0.25*|1110>


Check our calculation

In [6]:
s_truth = (
    1
    / sqrt(4)
    * (
        0.921 * Qubit(0, 0, 0, 1)
        + 0.390 * Qubit(0, 0, 1, 1)
        + 0.141 * Qubit(0, 1, 0, 0)
        + 0.99 * Qubit(0, 1, 1, 0)
        + 0.866 * Qubit(1, 0, 0, 1)
        + 0.500 * Qubit(1, 0, 1, 1)
        + 0.866 * Qubit(1, 1, 0, 0)
        + 0.500 * Qubit(1, 1, 1, 0)
    )
)
print(f"t={s_truth}")

assert simplify(s - s_truth) == 0

t=(0.921*|0001> + 0.39*|0011> + 0.141*|0100> + 0.99*|0110> + 0.866*|1001> + 0.5*|1011> + 0.866*|1100> + 0.5*|1110>)/2


Apply Hadamar to the fist qubit, quantum equivalent to coin flipping

In [7]:
# we apply Hadamard to the first qubit
g = IdentityGate(0) * IdentityGate(1) * IdentityGate(2) * HadamardGate(3)
x = qapply(g * s)
x

truth = (
    1
    / sqrt(8)
    * (
        (0.921 + 0.866) * Qubit(0, 0, 0, 1)
        + (0.390 + 0.500) * Qubit(0, 0, 1, 1)
        + (0.141 + 0.866) * Qubit(0, 1, 0, 0)
        + (0.990 + 0.500) * Qubit(0, 1, 1, 0)
        + (0.921 - 0.866) * Qubit(1, 0, 0, 1)
        + (0.390 - 0.500) * Qubit(1, 0, 1, 1)
        + (0.141 - 0.866) * Qubit(1, 1, 0, 0)
        + (0.990 - 0.500) * Qubit(1, 1, 1, 0)
    )
)
print(truth)
assert simplify(x - truth) == 0

sqrt(2)*(1.787*|0001> + 0.89*|0011> + 1.007*|0100> + 1.49*|0110> + 0.055*|1001> - 0.11*|1011> - 0.725*|1100> + 0.49*|1110>)/4


Measure first qubit, and keep measure where first qubit is zero

In [8]:
# measure the first qubit
measure = measure_partial(x, (3,))
measure = measure[0][0]  # get measure where first qubit is zero
print(measure)

0.470275598868077*sqrt(2)*|0001> + 0.234216722435696*sqrt(2)*|0011> + 0.265007010666006*sqrt(2)*|0100> + 0.392115636437288*sqrt(2)*|0110>


Calculate probabilities p0, p1

In [9]:
# p(q4 == 1 )
_, a = measure.as_independent(Qubit("0001"), as_Mul=True)
_, b = measure.as_independent(Qubit("0011"), as_Mul=True)
a = a / Qubit("0001")
b = b / Qubit("0011")

p0 = a * a + b * b
print(p0)

# p(q4 == 0 )
_, a = measure.as_independent(Qubit("0100"), as_Mul=True)
_, b = measure.as_independent(Qubit("0110"), as_Mul=True)
a = a / Qubit("0100")
b = b / Qubit("0110")

p1 = a * a + b * b
print(p1)

# matching results we found classicaly
assert np.abs(p0 - dists[0]) < 1e-2
assert np.abs(p1 - dists[1]) < 1e-2

0.552033223918495
0.447966776081505
